In [1]:
import pandas as pd
import plotly.express as px
import struct
import numpy as np
import math

def hex_to_float_q15(hex_string):
    # Convert the hexadecimal string to an unsigned integer.
    # The base 16 tells the int() function to interpret the string as hexadecimal.
    unsigned_int = int(hex_string, 16)

    # Normalize the integer to a float by dividing by the maximum 16-bit unsigned value.
    # We use 65535.0 to ensure the result is a float.
    return unsigned_int / 65535.0

def read_file(filename):
    float_values = []
    with open(filename, 'r') as f:
        for line in f:
            hex_val = line.strip()
            if hex_val:
                float_values.append(hex_to_float_q15(hex_val))
    return float_values       


In [2]:
lsb_value = 1.0 / 65535.0

df = pd.DataFrame({"ref":read_file("sineTable_lut.dat"), "gen":read_file("lut.mem")})
mse_manual = ((df["ref"] - df["gen"]) ** 2).mean()
df["rmse"] = np.sqrt(mse_manual)/lsb_value
df["err"] = abs(df["ref"] - df["gen"])/lsb_value
px.line(df)

In [49]:
def generate_reference(in_bits = 9):
    num_entries = 2**in_bits

    angle_step = (2 * math.pi) / (4 * num_entries)

    lut = list()

    for i in range(num_entries):
        
        value = math.cos(i * angle_step)
        
        lut.append(value)

    return lut

def generate_lut(in_bits = 9,out_bits = 16):

    num_entries = 2**in_bits

    angle_step = (2 * math.pi) / (4 * num_entries)

    lut = list()

    for i in range(num_entries):
        
        value = math.cos(i * angle_step)
   
        scaled_value = round(value * 2**out_bits)
        
        lut.append(scaled_value/(2**out_bits))

    return lut


def generate_interp_lut(in_bits=9, out_bits=16, factor=2):
    """
    Upsample LUT by 'factor' using only base LUT values.
    factor must be a power of two (2, 4, 8, ...).
    The last entry is forced to cos(pi/2) = 0.
    """
    base_lut = generate_lut(in_bits, out_bits)
    lut = []

    for i in range(len(base_lut) - 1):
        # base value
        lut.append(base_lut[i])
        # interpolated values
        for f in range(1, factor):
            interp_value = ((factor - f) * base_lut[i] + f * base_lut[i + 1]) / factor
            lut.append(interp_value)

    # final base entry (should be near 0) replaced with exact 0

    for i in range(0, factor):
        lut.append(lut[-1])

    lut[-1] = 0

    return lut
    
def lengthen_lut(lut, factor=2):
    out = []
    for i in range(len(lut) - 1):
        for j in range(0, factor):
            out.append(lut[i])
    
    for i in range(0, factor):
        out.append(lut[-1])
        
    return out

def get_lsb_error(meas, ref, n_bits=16):
    lsb_value = 1.0 / 2**n_bits
    return abs(meas - ref)/lsb_value


In [50]:
lut_9i = generate_interp_lut(9, 16)
lut_9p = lengthen_lut(generate_lut(9, 16))
lut_10 = generate_reference(10)

df = pd.DataFrame({"quantized": lut_9p, "interpolated": lut_9i, "native":lut_10})
df["q_err"] = get_lsb_error(df["quantized"], df["native"])
df["i_err"] = get_lsb_error(df["interpolated"], df["native"])

In [54]:
px.line(df)

In [59]:
lut_9i = generate_interp_lut(9, 16, factor=4)
lut_9i2 = lengthen_lut(generate_interp_lut(9, 16))
lut_9p = lengthen_lut(generate_lut(9, 16), factor=4)
lut_10 = generate_reference(11)

df = pd.DataFrame({"quantized": lut_9p, "quantized_interpolation": lut_9i2,  "interpolated": lut_9i, "native":lut_10})
df["q_err"] = get_lsb_error(df["quantized"], df["native"])
df["i2_err"] = get_lsb_error(df["quantized_interpolation"], df["native"])
df["i_err"] = get_lsb_error(df["interpolated"], df["native"])

In [60]:
px.line(df)

In [7]:
2**12

4096